time from timedata import

Target: 
PM2.5 :Airborne particulate matter (PM) is not a single pollutant, but rather is a mixture of many chemical species. It is a complex mixture of solids and aerosols composed of small droplets of liquid, dry solid fragments, and solid cores with liquid coatings. Particles vary widely in size, shape and chemical composition, and may contain inorganic ions, metallic compounds, elemental carbon, organic compounds, and compounds from the earth’s crust. Particles are defined by their diameter for air quality regulatory purposes. Those with a diameter of 10 microns or less (PM10) are inhalable into the lungs and can induce adverse health effects. Fine particulate matter is defined as particles that are 2.5 microns or less in diameter (PM2.5). Therefore, PM2.5 comprises a portion of PM10.

- Variation in size, shape and chemical composition
- contain inorganic ions, metallic compunds usw.
- Particles are defined by their diameter for air quality regulatory purposes
- diameter of 10 microns or less PM10 - are inhalable
- PM 2.5 microns or less in diameters -- portion of pm10


1. What is PM2.5
2. How does time matter and how do we split it?
3. What columns play a role to predict the target? 
4. NaN, Nulls, Data Cleaning, afterwards
5. can we add extra date from the internet?


The World Health Organization sets acceptable PM2.5 levels at 10 µg/m³ annually and at 25 µg/m³ in the 24-hour window.

its called forecasting


To understand the meaning of the columns in your dataset, you can take several approaches. Here are a few options:

1. Dataset Documentation or Data Dictionary
If your dataset is part of a larger project or publicly available, it may come with accompanying documentation or a data dictionary. This would explain each column and its significance. Look for any README files, documentation, or metadata that came with the dataset.

2. Column Naming Conventions
Sometimes column names can give you a clue about their meaning based on standard naming conventions. For example:

target: This could represent the dependent variable, likely air pollution concentration (e.g., PM2.5, NO2, etc.).
precipitable_water_entire_atmosphere: This might refer to the total water content in a vertical column of the atmosphere.
relative_humidity_2m_above_ground: Likely the relative humidity measured at 2 meters above the ground.
L3_NO2_NO2_column_number_density: Refers to the density of NO2 in a vertical atmospheric column.
These names often reflect the physical or environmental phenomena being measured, though understanding them fully might require some domain-specific knowledge (e.g., meteorology, atmospheric science).

3. Consult with the Data Provider or Use External Resources
If you obtained this data from an organization (e.g., NASA, a governmental agency, or a scientific repository), there is often accompanying documentation on their website or in related publications that describe the columns.

For scientific datasets, you might need to reference papers or studies that provide detailed explanations about how the data is collected and what the specific metrics mean.

4. Investigate Common Terms
If the dataset includes environmental, meteorological, or pollutant-specific columns, here are a few general references you can look up:

NO2, CO, SO2, O3, CH4: These are common air pollutants (Nitrogen Dioxide, Carbon Monoxide, Sulfur Dioxide, Ozone, Methane, etc.), and columns like L3_NO2_NO2_column_number_density might indicate concentrations of these substances.
Humidity and Wind Components: Terms like "humidity" and "wind component" are related to atmospheric conditions, with columns likely measuring these at specific heights or in specific orientations.

group by different location and create differend panda frames for it, and for each of them we create a sub_df and target, and we do .shift(1) what it will do it will shift by one, it will give us yesterdays value if we shift by one

for that we can create new variable, sub df leg 1 

we create a new column, feature engineering, where for each place, what is the observation of the day before, but before this we need to do the train test split, it should also be in a forward looking fashion, take the last three days for each location as test set, in a forecasting problem we dont do long term prediction, its okay to take the alst 3 days as test set. Its gonna be location based. 

What are the two? create yesterday observation as a feature with groupby magic
generic thing create test set, forward looking test, last three days, it needs also group by manipulation


MAPE mean absolute error 


train set has 
day 1 - day 60

and test set

day 61, 62, 63

we need to add column which location, some are target Place_ID  - NEW column lag 1 --> day before (once created we do XGBOOST or linear regression on it - time aspect), first we create lage 1

sub df for each location as part of the est set

pearson correlation can be computed even with missing values, to see which features are not so powerful - for feature selection! 

measure performance, other complexity

how do we feed sub dfs in regression?

Baseline model lag1 feature - what is the target of the day before - drop the dates where no date before is (like first date) - lag 1 is corresponding to its own place id, so than the date problem is gone and we can feed it in the regression

linear regression (Xtrain, Ytrain - polluted yesterday?

lag 1 on our test data is the prediction but not the ground root! - to build approximation on approximation is - time series problem extremely noisy - tmrw becomes new data, continuesoly predict on for one day

In [ ]:
# Step 1: Custom Transformer for Group-Wise Imputation
class GroupImputer(BaseEstimator, TransformerMixin):
    def __init__(self, group_col, strategy='mean'):
        self.group_col = group_col
        self.strategy = strategy
    
    def fit(self, X, y=None):
        # Separate the group column from the features
        self.columns_ = X.columns.drop(self.group_col)
        
        # Compute group-wise statistics (mean) for each numerical feature
        self.impute_values_ = X.groupby(self.group_col).transform(lambda x: x.mean())
        return self
    
    def transform(self, X):
        # Fill missing values based on group statistics
        X_filled = X.copy()
        for col in self.columns_:
            # Use the group-wise mean for each numerical feature
            X_filled[col] = X_filled[col].fillna(self.impute_values_[col])
        return X_filled.drop(columns=[self.group_col])

# Step 2: Define numerical and categorical features
numerical_features = ["L3_CO_CO_column_number_density", 
                      "L3_HCHO_tropospheric_HCHO_column_number_density",
                      "L3_NO2_NO2_column_number_density", 
                      "L3_O3_O3_column_number_density", 
                      "u_component_of_wind_10m_above_ground"]

categorical_features = ['Place_ID']  # Ensure 'Place_ID' exists in your DataFrame

# Step 3: Preprocess the data (Group-Wise Imputation + Fallback Imputation + Scaling/Encoding)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('group_imputer', GroupImputer(group_col='Place_ID')),  # Custom group-wise imputation
            ('fallback_imputer', SimpleImputer(strategy='mean')),  # Fallback imputer for any remaining NaNs
            ('scaler', StandardScaler())  # Standardize the numerical features
        ]), numerical_features + ['Place_ID']),  # Pass Place_ID for group imputation
        ('cat', OneHotEncoder(drop='first'), categorical_features)  # OneHotEncode Place_ID
    ])

# Define the target variable
y_train = df_train_split["target"]
X_train = df_train_split[numerical_features + categorical_features]

y_test = df_test_split["target"]
X_test = df_test_split[numerical_features + categorical_features]

# Step 6: Create a pipeline with preprocessing and linear regression
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Step 7: Fit the model
pipeline.fit(X_train, y_train)

# Step 8: Make predictions and evaluate the model
y_pred = pipeline.predict(X_test)

# Step 9: Evaluate the performance
mape = mean_absolute_percentage_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Percentage Error (MAPE): {mape :.2f}%')
print(f'R² Score: {r2}')

1. Missing Data:
You have many features with missing data, some of which have a significant portion of missing values. For example:

L3_NO2_NO2_column_number_density and other NO2-related features have 2186 missing values.
L3_CH4_CH4_column_volume_mixing_ratio_dry_air and other CH4-related features have 24162 missing values.
Large portions of missing data can cause issues with model performance, especially if the missing values are not imputed correctly. Although you're using mean imputation, if the missing data is not random, it could introduce bias.

Action: Consider a more sophisticated imputation strategy, such as KNN imputation, or drop features with too many missing values. Alternatively, try models that handle missing data natively, such as XGBoost or LightGBM.

2. Outliers:
You have a significant number of outliers in your numerical features, particularly:

u_component_of_wind_10m_above_ground has 1413 outliers.
L3_NO2_NO2_slant_column_number_density has 3567 outliers.
L3_CLOUD_cloud_optical_depth has 2721 outliers.
Outliers can heavily skew models like linear regression, leading to poor performance.

Action:

You may want to cap (winsorize) or remove outliers from your dataset.
Alternatively, use robust models that are less sensitive to outliers (e.g., Huber Regressor, Gradient Boosting with outlier handling).
3. Low Variance Features:
You've identified several low-variance features that may not provide much information for the model. These include:

L3_NO2_NO2_slant_column_number_density
L3_CO_CO_column_number_density
L3_SO2_SO2_slant_column_number_density
Low-variance features add little value and may increase noise in the model.

Action: Drop these low-variance features from your dataset.

4. Collinearity:
Some of your features are likely highly correlated (multicollinearity), especially since the NO2, O3, and CO-related features are derived from similar sources. Multicollinearity can cause problems with linear models, leading to inflated errors.

Action: Use a correlation matrix to remove highly correlated features or use regularization techniques like Ridge or Lasso regression to mitigate this issue.

5. Scaling Issues:
If your numerical features are not properly scaled, this can cause issues with models like linear regression and gradient boosting.

Action: Ensure that your numerical features are standardized (mean = 0, std = 1) or normalized (range 0 to 1), depending on the model you're using.

6. Data Type Issues:
There don't seem to be any apparent data type issues from your check. However, ensure that the categorical features are properly encoded before feeding them into the model.



In [ ]:
# Group by 'Place_ID' and apply imputation within each group
def impute_group(group):
    # Example imputation: fill NaN with the mean or median for each column within the group
    group.fillna(group.mean(), inplace=True)  # Or use group.median() or forward fill (group.ffill())
    return group

# Apply the imputation function to each group
df_imputed = df_sorted.groupby('Place_ID').apply(impute_group)

# If necessary, you can re-sort the imputed data again by 'Place_ID' and 'Date'
df_imputed = df_imputed.sort_values(by=['Place_ID', 'Date'])

# Optional: Check if there are still any NaNs left
remaining_nans = df_imputed.isna().sum()

# Display the remaining NaNs
print("Remaining NaNs after imputation:\n", remaining_nans)

In [1]:
Summary of Features and Imputation Strategies
target
Count: Complete data (no missing values).
Strategy: No imputation needed.
precipitable_water_entire_atmosphere
Count: Complete data.
Strategy: Mean imputation (normally distributed).
relative_humidity_2m_above_ground
Count: Complete data.
Strategy: Mean imputation (typically normal distribution).
specific_humidity_2m_above_ground
Count: Complete data.
Strategy: Mean imputation (normal variation expected).
temperature_2m_above_ground
Count: Complete data.
Strategy: Mean imputation (central tendency is important).
u_component_of_wind_10m_above_ground
Count: Complete data.
Strategy: Mean imputation (normally distributed wind components).
v_component_of_wind_10m_above_ground
Count: Complete data.
Strategy: Mean imputation (same as u-component).
L3_NO2_NO2_column_number_density
Count: 28,348 (2,187 missing).
Strategy: Median imputation (likely skewed due to pollution spikes).
L3_NO2_NO2_slant_column_number_density
Count: 28,348 (2,187 missing).
Strategy: Median imputation (outliers present).
L3_NO2_absorbing_aerosol_index
Count: 28,348 (2,187 missing).
Strategy: Median imputation (sensitive to extreme values).
L3_NO2_cloud_fraction
Count: 28,348 (2,187 missing).
Strategy: Median imputation (typically binary presence).
L3_NO2_sensor_altitude
Count: 28,348 (2,187 missing).
Strategy: Median imputation (altitude values may be clustered).
L3_NO2_sensor_azimuth_angle
Count: 28,347 (2,188 missing).
Strategy: Median imputation (skewed data requires median).
L3_NO2_sensor_zenith_angle
Count: 28,347 (2,188 missing).
Strategy: Median imputation (protects against outliers).
L3_NO2_solar_azimuth_angle
Count: 28,347 (2,188 missing).
Strategy: Median imputation (similar reasoning).
L3_NO2_solar_zenith_angle
Count: 28,347 (2,188 missing).
Strategy: Median imputation (avoids skew).
L3_NO2_stratospheric_NO2_column_number_density
Count: 28,347 (2,188 missing).
Strategy: Median imputation (high variability).
L3_NO2_tropopause_pressure
Count: 28,347 (2,188 missing).
Strategy: Median imputation (pressure can vary widely).
L3_NO2_tropospheric_NO2_column_number_density
Count: 21,870 (8,665 missing).
Strategy: Median imputation (extreme values).
L3_O3_O3_column_number_density
Count: 30,253 (missing values).
Strategy: Median imputation (affected by environmental factors).
L3_O3_O3_effective_temperature
Count: 30,253 (missing values).
Strategy: Median imputation (can be skewed).
L3_O3_cloud_fraction
Count: 30,253 (missing values).
Strategy: Median imputation (central measure is important).
L3_O3_sensor_azimuth_angle
Count: 30,253 (missing values).
Strategy: Median imputation (outliers can skew results).
L3_O3_sensor_zenith_angle
Count: 30,253 (missing values).
Strategy: Median imputation (similar reasoning).
L3_O3_solar_azimuth_angle
Count: 30,253 (missing values).
Strategy: Median imputation (protect against extremes).
L3_O3_solar_zenith_angle
Count: 30,253 (missing values).
Strategy: Median imputation (avoids skew).
L3_CO_CO_column_number_density
Count: 25,114 (5,421 missing).
Strategy: Median imputation (pollution variability).
L3_CO_H2O_column_number_density
Count: 25,114 (5,421 missing).
Strategy: Median imputation (extreme values may exist).
L3_CO_cloud_height
Count: 25,114 (5,421 missing).
Strategy: Median imputation (high variability).
L3_CO_sensor_altitude
Count: 25,114 (5,421 missing).
Strategy: Median imputation (clustered values).
L3_CO_sensor_azimuth_angle
Count: 25,114 (5,421 missing).
Strategy: Median imputation (affected by outliers).
L3_CO_sensor_zenith_angle
Count: 25,114 (5,421 missing).
Strategy: Median imputation (skewed distribution).
L3_CO_solar_azimuth_angle
Count: 25,114 (5,421 missing).
Strategy: Median imputation (similar reasoning).
L3_CO_solar_zenith_angle
Count: 25,114 (5,421 missing).
Strategy: Median imputation (protect against extremes).
L3_HCHO_HCHO_slant_column_number_density
Count: 23,115 (7,420 missing).
Strategy: Median imputation (environmental influences).
L3_HCHO_cloud_fraction
Count: 23,115 (7,420 missing).
Strategy: Median imputation (central measure is important).
L3_HCHO_sensor_azimuth_angle
Count: 23,115 (7,420 missing).
Strategy: Median imputation (outliers can skew results).
L3_HCHO_sensor_zenith_angle
Count: 23,115 (7,420 missing).
Strategy: Median imputation (avoids skew).
L3_HCHO_solar_azimuth_angle
Count: 23,115 (7,420 missing).
Strategy: Median imputation (protect against extremes).
L3_HCHO_solar_zenith_angle
Count: 23,115 (7,420 missing).
Strategy: Median imputation (similar reasoning).
L3_HCHO_tropospheric_HCHO_column_number_density
Count: 23,115 (7,420 missing).
Strategy: Median imputation (high variability).
L3_HCHO_tropospheric_HCHO_column_number_density_amf
Count: 23,115 (7,420 missing).
Strategy: Median imputation (high variability).
L3_CLOUD_cloud_base_height
Count: 29,014 (5,521 missing).
Strategy: Median imputation (skewed data expected).
L3_CLOUD_cloud_base_pressure
Count: 29,014 (5,521 missing).
Strategy: Median imputation (can vary widely).
L3_CLOUD_cloud_optical_depth
Count: 29,014 (5,521 missing).
Strategy: Median imputation (sensitive to extreme values).
L3_CLOUD_cloud_top_height
Count: 29,014 (5,521 missing).
Strategy: Median imputation (similar reasoning).
L3_CLOUD_cloud_top_pressure
Count: 29,014 (5,521 missing).
Strategy: Median imputation (high variability).
L3_CLOUD_surface_albedo
Count: 29,014 (5,521 missing).
Strategy: Median imputation (skewed data expected).
L3_AER_AI_absorbing_aerosol_index
Count: 30,339 (196 missing).
Strategy: Median imputation (outlier protection).
L3_AER_AI_sensor_altitude
Count: 30,339 (196 missing).
Strategy: Median imputation (clustered values).
L3_AER_AI_sensor_azimuth_angle
Count: 30,339 (196 missing).
Strategy: Median imputation (affected by outliers).
L3_AER_AI_sensor_zenith_angle
Count: 30,339 (196 missing).
Strategy: Median imputation (avoids skew).
L3_AER_AI_solar_azimuth_angle
Count: 30,339 (196 missing).
Strategy: Median imputation (protect against extremes).
L3_AER_AI_solar_zenith_angle
Count: 30,339 (196 missing).
Strategy: Median imputation (similar reasoning).
L3_SO2_SO2_column_number_density
Count: 23,304 (7,231 missing).
Strategy: Median imputation (high variability).
L3_SO2_SO2_column_number_density_amf
Count: 23,304 (7,231 missing).
Strategy: Median imputation (high variability).
L3_SO2_SO2_slant_column_number_density
Count: 23,304 (7,231 missing).
Strategy: Median imputation (sensitive to extreme values).
L3_SO2_absorbing_aerosol_index
Count: 23,233 (7,302 missing).
Strategy: Median imputation (outlier protection).
L3_SO2_cloud_fraction
Count: 23,304 (7,231 missing).
Strategy: Median imputation (central measure is important).
L3_SO2_sensor_azimuth_angle
Count: 23,304 (7,231 missing).
Strategy: Median imputation (affected by outliers).
L3_SO2_sensor_zenith_angle
Count: 23,304 (7,231 missing).
Strategy: Median imputation (avoids skew).
L3_SO2_solar_azimuth_angle
Count: 23,304 (7,231 missing).
Strategy: Median imputation (protect against extremes).
L3_SO2_solar_zenith_angle
Count: 23,304 (7,231 missing).
Strategy: Median imputation (similar reasoning).


SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (1192019511.py, line 129)

8. Check for Data Leakage:
Make sure that your test data is not being contaminated with information from the training set. For example, ensure that lag features are calculated only from the past data, not future data.

Conclusion:
Start by testing more complex models like Random Forest, Gradient Boosting, or XGBoost.
Improve feature engineering by adding lag features, rolling averages, and interaction terms.
Use log transformation to handle skewness in the target.
Regularization and ensemble techniques can also help prevent overfitting.
Always validate your results using cross-validation or a validation set.
By applying these strategies, you should see significant improvement in your RMSE scores!